In [19]:
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import pandas as pd
import ast
import requests
from io import BytesIO

parquet_fname = '../data/laion2b-en-1K-experts-large.parquet'
df = pd.read_parquet(parquet_fname, engine='pyarrow')

In [20]:
df

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity,OCR_BBOXES,OCR_EXPERTS,OCR_TEXT
0,2.641080e+12,https://cdn.shopify.com/s/files/1/0017/3621/25...,"Blue Beach Umbrellas, Point Of Rocks, Crescent...",231.0,300.0,?,UNLIKELY,0.395562,[],[],[]
1,1.069682e+12,http://cdn.bmwblog.com/wp-content/uploads/2016...,BMW-M2-M-Performance-Dekor-Long-Beach-Blue-05,120.0,120.0,?,UNLIKELY,0.324732,[],[],[]
2,7.480780e+11,http://ecx.images-amazon.com/images/I/51phsP4a...,Becoming More Than a Good Bible Study Girl: Li...,125.0,125.0,?,UNLIKELY,0.425868,"[[[46.0, 339.0], [470.0, 339.0], [470.0, 394.0...","['trocr-large-stage1', 'trocr-large-stage1', '...","['BECOMING MORE', 'THAN A GOOD BIBLE STUDY CIR..."
3,3.203431e+12,https://www.deltekproducts.com/_resources/imag...,"""Dynabrade 52632 4-1/2"""" Dia. Right Angle Depr...",150.0,100.0,?,UNLIKELY,0.310137,[],[],[]
4,1.517065e+12,http://www.gamersnine.com/file_g9/2014/05/MANE...,MANETTE XBOX ONE,650.0,431.0,?,NSFW,0.317398,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
996,3.253112e+12,https://img-aws.ehowcdn.com/137x77/photos.dema...,How to Open a Hairdressing Salon,137.0,77.0,?,UNLIKELY,0.301545,[],[],[]
997,4.119222e+12,https://static.wixstatic.com/media/1c009e_89d1...,5Pc Set Florian Dining Table w/ 4 Chairs,498.0,331.0,?,UNLIKELY,0.339066,[],[],[]
998,1.525458e+12,http://cache.boston.com/bonzai-fba/Globe_Photo...,mounted police,508.0,334.0,?,UNLIKELY,0.311321,[],[],[]
999,1.831123e+12,https://id-live-03.slatic.net/p/4/image-878256...,3 in 1 - Diaper Bag - Travel Bassinet - Change...,340.0,340.0,?,UNLIKELY,0.317100,[],[],[]


In [21]:
def parse_list_string(list_string):
    return list_string.strip('[]').replace("'", "").split(',')

# Parse strings into lists and extract unique strings
unique_strings = set(
    item for sublist in df['OCR_EXPERTS'].apply(parse_list_string) for item in sublist
)


In [22]:
exp_col = df['OCR_EXPERTS']

In [23]:
handwritten_idcs = []
for idx, exps in enumerate(exp_col):
    #if 'paddleocr' in exps:
    if f'trocr-large-handwritten' in exps:
        handwritten_idcs += [idx]

In [24]:
def on_click(idx):
    with output:
        clear_output(wait=True)
        display(Image(url=df.iloc[idx]['URL']))
        ocr_texts = parse_list_string(df.iloc[idx]['OCR_TEXT'])
        ocr_experts = parse_list_string(df.iloc[idx]['OCR_EXPERTS'])
        for ocr_text, ocr_experts in zip(ocr_texts, ocr_experts):
            print("{0: <40} {1: <40}".format(ocr_text, ocr_experts))
    
def url_to_memory_view(url):
    response = requests.get(url)
    img_data = BytesIO(response.content)
    return memoryview(img_data.getvalue())

def on_thumbnail_click(idx, button):
    on_click(idx)

In [25]:
buttons = []
for i, idx in enumerate(handwritten_idcs):
    button = widgets.Button(description='', icon='check', layout=widgets.Layout(width='100px', height='100px'))
    button.style.button_color = 'transparent'
    button.on_click(lambda btn, idx=idx: on_thumbnail_click(idx, btn))
    buttons.append(button)

thumbnails = [widgets.HTML(f'<img src="{df.iloc[idx]["URL"]}" width="100" height="100"/>') for idx in handwritten_idcs]

thumbnails_grid = widgets.GridBox([widgets.VBox([thumbnail, button]) for thumbnail, button in zip(thumbnails, buttons)], layout=widgets.Layout(grid_template_columns="repeat(auto-fill, minmax(100px, 1fr))", grid_gap="10px"))

output = widgets.Output()

main_layout = widgets.VBox([thumbnails_grid, output])
display(main_layout)